# 9 - Metaprogramming

## Putting a Wrapper Around a Function

In [1]:
import time
from functools import wraps

def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        result = func(*args, **kwargs)
        end = time.time()
        print(func.__name__, end-start)
        return result
    return wrapper


In [2]:
@timethis
def countdown(n):
    while n > 0:
        n -= 1


In [5]:
countdown(1e6)

countdown 0.07303404808044434


## Preserving Function Metadata When Writing Decorators
Remember to use @wraps...

In [8]:
def action(func):
    def wrapper(*args, **kwargs):
        ''' Docs for wrapper '''
        result = func(*args, **kwargs)
        return result
    return wrapper

@action
def adder(a, b):
    ''' Docs for adder '''
    return a + b

adder(1, 2)

3

In [11]:
adder.__name__

'wrapper'

In [13]:
adder.__doc__

' Docs for wrapper '

In [14]:
adder.__annotations__

{}

If we use @wraps instead...

In [1]:
from functools import wraps

def action(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        ''' Docs for wrapper '''
        result = func(*args, **kwargs)
        return result
    return wrapper

@action
def adder(a, b):
    ''' Docs for adder '''
    return a + b

adder(1, 2)

3

In [17]:
adder.__name__

'adder'

In [18]:
adder.__doc__

' Docs for adder '

In [19]:
adder.__annotations__

{}

## Unwrapping a Decorator
If you have previously used @wraps

In [5]:
from functools import wraps

def offbyone(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        ''' Docs for wrapper '''
        result = func(*args, **kwargs) + 1
        return result
    return wrapper

@offbyone
def adder(a, b):
    ''' Docs for adder '''
    return a + b


adder(1, 2)

4

In [6]:
adder

<function __main__.adder(a, b)>

In [7]:
orig_add = adder.__wrapped__
orig_add(1, 2)

3

## Defining a Decorator That Takes Arguments

In [8]:
from functools import wraps
import logging

def logged(level, name=None, message=None):
    '''
    Add logging to a function. level is the logging
    level, name is the logger name, and message is the
    log message. If name and message aren't specified,
    they default to the function's module and name.
    '''
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__
 
        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)
    
        return wrapper
    
    return decorate


In [9]:
@logged(logging.DEBUG)
def add(x, y):
    return x + y

add(1,2)

3

## Defining a Decorator with User Adjustable Attributes

In [10]:
from functools import wraps, partial
import logging


# utility decorator to attach a function as an attribute of obj
def attach_wrapper(obj, func=None):
    if func is None:
        return partial(attach_wrapper, obj)
    setattr(obj, func.__name__, func)
    return func


In [12]:
def logged(level, name=None, message=None):
    '''
    Add logging to a function. level is the logging
    level, name is the logger name, and message is the
    log message. If name and message aren't specified,
    they default to the function's module and name.
    '''
    def decorate(func):
        logname = name if name else func.__module__
        log = logging.getLogger(logname)
        logmsg = message if message else func.__name__
 
        @wraps(func)
        def wrapper(*args, **kwargs):
            log.log(level, logmsg)
            return func(*args, **kwargs)

         # attach setter functions
        @attach_wrapper(wrapper)
        def set_level(newlevel):
            nonlocal level
            level = newlevel

        @attach_wrapper(wrapper)
        def set_message(newmsg):
            nonlocal logmsg
            logmsg = newmsg

        return wrapper
    return decorate


In [13]:
@logged(logging.DEBUG)
def add(x, y):
    return x + y

In [14]:
import logging

logging.basicConfig(level=logging.DEBUG)
add(2,3)

DEBUG:__main__:add


5

In [15]:
add.set_message('Add called')
add(2,3)

DEBUG:__main__:Add called


5

In [16]:
add.set_level(logging.WARNING)
add(2,3)

5

## Enforcing Type Checking on a Function Using a Decorator

In [17]:
from inspect import signature
from functools import wraps

def typeassert(*ty_args, **ty_kwargs):
    def decorate(func):
        # If in optimized mode, disable type checking
        if not __debug__:
            return func
 
        # Map function argument names to supplied types
        sig = signature(func)
        bound_types = sig.bind_partial(*ty_args, **ty_kwargs).arguments
 
        @wraps(func)
        def wrapper(*args, **kwargs):
            bound_values = sig.bind(*args, **kwargs)
            # Enforce type assertions across supplied arguments
            for name, value in bound_values.arguments.items():
                if name in bound_types:
                    if not isinstance(value, bound_types[name]):
                        raise TypeError('Argument {} must be {}'.format(name, bound_types[name]))
            return func(*args, **kwargs)
 
        return wrapper
    return decorate


In [18]:
@typeassert(int, z=int)
def spam(x, y, z=42):
    print(x, y, z)


In [19]:
spam(1,2,3)

1 2 3


In [20]:
spam(1, 'hello', 3)

1 hello 3


In [21]:
spam(1, 'hello', 'world')

TypeError: Argument z must be <class 'int'>

In [22]:
__debug__

True

In [25]:
import inspect

sig = inspect.signature(spam)
sig

<Signature (x, y, z=42)>

In [26]:
sig.parameters

mappingproxy({'x': <Parameter "x">,
              'y': <Parameter "y">,
              'z': <Parameter "z=42">})

In [27]:
sig.parameters['z'].name

'z'

In [28]:
sig.parameters['z'].default

42

In [29]:
sig.parameters['z'].kind

<_ParameterKind.POSITIONAL_OR_KEYWORD: 1>

In [31]:
bound_types = sig.bind_partial(int,z=int)

In [32]:
bound_types

<BoundArguments (x=<class 'int'>, z=<class 'int'>)>

In [33]:
bound_types.arguments  # the missing arguments are simply ignored

OrderedDict([('x', int), ('z', int)])

## Defining Decorators As Part of a Class

In [35]:
from functools import wraps

class A:
    # decorator as an instance method
    def decorator1(self, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print('Decorator 1')
            return func(*args, **kwargs)
        return wrapper
 
    # decorator as a class method
    @classmethod
    def decorator2(cls, func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            print('Decorator 2')
            return func(*args, **kwargs)
        return wrapper


In [36]:
a = A()
@a.decorator1  # applied from the instance a
def spam():
    pass


@A.decorator2  # applied from the class A
def grok():
    pass


In [37]:
spam()

Decorator 1


In [38]:
grok()

Decorator 2


## Defining Decorators As Classes
To define a decorator as an instance, you need to make sure it implements the __call__() and __get__() methods. 

In [39]:
import types
from functools import wraps

class Profiled:
    def __init__(self, func):
        wraps(func)(self)
        self.ncalls = 0
 
    def __call__(self, *args, **kwargs):
        self.ncalls += 1
        return self.__wrapped__(*args, **kwargs)
 
    def __get__(self, instance, cls):
        if instance is None:
            return self
        else:
            return types.MethodType(self, instance)


To use this class, you use it like a normal decorator, either inside or outside of a class.

In [40]:
@Profiled
def add(x, y):
    return x + y

class Spam:
    @Profiled
    def bar(self, x):
        print(self, x)


In [41]:
add(1,2)

3

In [42]:
s = Spam()
s.bar(1)

<__main__.Spam object at 0x00000204A6315438> 1


In [43]:
add.ncalls

1

In [44]:
add(2,3)
add.ncalls

2

In [45]:
Spam.bar.ncalls

1

In [46]:
s.bar(2)
Spam.bar.ncalls

<__main__.Spam object at 0x00000204A6315438> 2


2

In [48]:
vars(Profiled)

mappingproxy({'__module__': '__main__',
              '__init__': <function __main__.Profiled.__init__(self, func)>,
              '__call__': <function __main__.Profiled.__call__(self, *args, **kwargs)>,
              '__get__': <function __main__.Profiled.__get__(self, instance, cls)>,
              '__dict__': <attribute '__dict__' of 'Profiled' objects>,
              '__weakref__': <attribute '__weakref__' of 'Profiled' objects>,
              '__doc__': None})

In [54]:
class A:
    pass

x = A()
y = A()

In [55]:
A.__dict__

mappingproxy({'__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None})

In [57]:
A.__weakref__.__doc__

'list of weak references to the object (if defined)'

In [60]:
#dir(A.__weakref__)

## Applying Decorators to Class and Static Methods
Make sure that your decorators are applied after @classmethod or @staticmethod.

In [1]:
import time
from functools import wraps

# a simple decorator
def timethis(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        start = time.time()
        r = func(*args, **kwargs)
        end = time.time()
        print(end-start)
        return r
    return wrapper


In [2]:
# Class illustrating application of the decorator to different kinds of methods
class Spam:
    @timethis
    def instance_method(self, n):
        print(self, n)
        while n > 0:
            n -= 1

    @classmethod
    @timethis
    def class_method(cls, n):
        print(cls, n)
        while n > 0:
            n -= 1

    @staticmethod
    @timethis
    def static_method(n):
        print(n)
        while n > 0:
            n -= 1


In [6]:
s = Spam()
s.instance_method(1e6)    

<__main__.Spam object at 0x000001AC37A79FD0> 1000000.0
0.10300064086914062


In [7]:
s.class_method(1e6) 

<class '__main__.Spam'> 1000000.0
0.10799980163574219


In [8]:
s = Spam()
s.static_method(1e6) 

1000000.0
0.06400036811828613


## Writing Decorators That Add Arguments to Wrapped Functions

Extra arguments can be injected into the calling signature using keyword-only arguments.

In [10]:
from functools import wraps

def optional_debug(func):
    @wraps(func)
    def wrapper(*args, debug=False, **kwargs):
        if debug:
            print('Calling', func.__name__)
        return func(*args, **kwargs)
    return wrapper


In [11]:
@optional_debug
def spam(a,b,c):
    print(a,b,c)


In [12]:
spam(1,2,3)

1 2 3


In [13]:
spam(1,2,3, debug=True)

Calling spam
1 2 3


if the @optional_debug decorator was applied to a function that already had a debug argument, then it would break. If that’s a concern, then we can add extra checking for it.

## Using Decorators to Patch Class Definitions
Class decorators can often be used as a straightforward alternative to other more advanced techniques involving mixins or metaclasses.


In [14]:
def log_getattribute(cls):
    # Get the original implementation
    orig_getattribute = cls.__getattribute__
 
    # Make a new definition
    def new_getattribute(self, name):
        print('getting:', name)
        return orig_getattribute(self, name)
 
    # Attach to the class and return
    cls.__getattribute__ = new_getattribute
    return cls


In [15]:
@log_getattribute
class A:
    def __init__(self,x):
        self.x = x
 
    def spam(self):
        pass


In [16]:
a = A(42)

In [17]:
a.x

getting: x


42

In [18]:
a.spam()

getting: spam


## Using a Metaclass to Control Instance Creation

In [19]:
class NoInstances(type):
    def __call__(self, *args, **kwargs):
        raise TypeError("Can't instantiate directly")


In [20]:
class Spam(metaclass=NoInstances):
    @staticmethod
    def grok(x):
        print('Spam.grok')


In [22]:
Spam.grok(1)

Spam.grok


In [23]:
s = Spam()

TypeError: Can't instantiate directly

Now, suppose you want to implement the singleton pattern.

In [1]:
class Singleton(type):
    def __init__(self, *args, **kwargs):
        self.__instance = None
        super().__init__(*args, **kwargs)
 
    def __call__(self, *args, **kwargs):
        if self.__instance is None:
            self.__instance = super().__call__(*args, **kwargs)
            return self.__instance
        else:
            return self.__instance


In [2]:
class Spam(metaclass=Singleton):
    def __init__(self):
        print('Creating Spam')


In [3]:
a = Spam()

Creating Spam


In [4]:
b = Spam()

In [5]:
a is b

True

Or suppose you want to create cached instances.

In [7]:
# basically caching, but also see flyweight pattern.
import weakref

class Cached(type):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.__cache = weakref.WeakValueDictionary()
 
    def __call__(self, *args):
        if args in self.__cache:
            return self.__cache[args]
        else:
            obj = super().__call__(*args)
            self.__cache[args] = obj
            return obj


In [8]:
class Spam(metaclass=Cached):
    def __init__(self, name):
        print('Creating Spam({!r})'.format(name))
        self.name = name


In [9]:
a = Spam('Guido')

Creating Spam('Guido')


In [10]:
b = Spam('Diana')

Creating Spam('Diana')


In [11]:
c = Spam('Guido')  # cached

In [13]:
a is c

True

In [14]:
a is b

False

## Capturing Class Attribute Definition Order
You want to automatically record the order in which attributes and methods are defined inside a class body so that you can use it in various operations.

In [1]:
from collections import OrderedDict

# A set of descriptors for various types
class Typed:
    _expected_type = type(None)
    def __init__(self, name=None):
        self._name = name
 
    def __set__(self, instance, value):
        if not isinstance(value, self._expected_type):
            raise TypeError('Expected ' + str(self._expected_type))
        instance.__dict__[self._name] = value

class Integer(Typed):
    _expected_type = int

class Float(Typed):
    _expected_type = float

class String(Typed):
    _expected_type = str


In [2]:
class OrderedMeta(type):
    def __new__(cls, clsname, bases, clsdict):
        d = dict(clsdict)
        order = []
        for name, value in clsdict.items():
            if isinstance(value, Typed):
                value._name = name
                order.append(name)
        d['_order'] = order
        return type.__new__(cls, clsname, bases, d)
 
    @classmethod
    def __prepare__(cls, clsname, bases):
        return OrderedDict()


The [__prepare__()](https://python-3-patterns-idioms-test.readthedocs.io/en/latest/Metaprogramming.html#the-prepare-metamethod) method is interesting and expanded upon in [this article](https://www.artima.com/weblogs/viewpost.jsp?thread=236260).

In [3]:
class Structure(metaclass=OrderedMeta):
    def as_csv(self):
        return ','.join(str(getattr(self,name)) for name in self._order)


In [4]:
class Stock(Structure):
    name = String()
    shares = Integer()
    price = Float()
    def __init__(self, name, shares, price):
        self.name = name
        self.shares = shares
        self.price = price


In [5]:
s = Stock('GOOG',100,490.1)

In [6]:
s.name

'GOOG'

In [8]:
s.as_csv()

'GOOG,100,490.1'

The entire key to this recipe is the __prepare__() method, which is defined in the OrderedMeta metaclass. This method is invoked immediately at the start of a class definition with the class name and base classes.

## Defining a Metaclass That Takes Optional Arguments
In the case of ABCs

In [9]:
from abc import ABCMeta, abstractmethod

class IStream(metaclass=ABCMeta):
    @abstractmethod
    def read(self, maxsize=None):
        pass
 
    @abstractmethod
    def write(self, data):
        pass


And for a custom metaclass.

In [13]:
class MyMeta(type):
    @classmethod
    def __prepare__(cls, name, bases, *, debug=False, synchronize=False):
        # Custom processing ...
        return super().__prepare__(name, bases)
 
    # Required
    def __new__(cls, name, bases, ns, *, debug=False, synchronize=False):
        # Custom processing ...
        return super().__new__(cls, name, bases, ns)
 
    # Required
    def __init__(self, name, bases, ns, *, debug=False, synchronize=False):
        # Custom processing ...
        super().__init__(name, bases, ns)


class Spam(metaclass=MyMeta, debug=True, synchronize=True):
    pass


## Enforcing an Argument Signature on *args and **kwargs

In [1]:
from inspect import Signature, Parameter

In [2]:
parms = [Parameter('x', Parameter.POSITIONAL_OR_KEYWORD),
         Parameter('y', Parameter.POSITIONAL_OR_KEYWORD, default=42),
         Parameter('z', Parameter.KEYWORD_ONLY, default=None)]

sig = Signature(parms)
sig

<Signature (x, y=42, *, z=None)>

Once you have a signature object, you can easily bind it to *args and **kwargs using the signature’s bind() method.

In [3]:
def func(*args, **kwargs):
    bound_values = sig.bind(*args, **kwargs)
    for name, value in bound_values.arguments.items():
        print(name,value)


In [4]:
func(1, 2, z=3)

x 1
y 2
z 3


In [5]:
func(1, 2, 3, 4)

TypeError: too many positional arguments

## Enforcing Coding Conventions in Classes
If you want to monitor the definition of classes, you can often do it by defining a metaclass. A basic metaclass is usually defined by inheriting from type and redefining its __new__() method or __init__() method.

In [1]:
class MyMeta(type):
    def __new__(self, clsname, bases, clsdict):
        # clsname is name of class being defined
        # bases is tuple of base classes
        # clsdict is class dictionary
        return super().__new__(cls, clsname, bases, clsdict)


In [2]:
# if init is defined
class MyMeta(type):
    def __init__(self, clsname, bases, clsdict):
        super().__init__(clsname, bases, clsdict)


You would generally incorporate it into a top-level base class from which other objects inherit.

In [3]:
class Root(metaclass=MyMeta):
    pass

class A(Root):
    pass

class B(Root):
    pass


As a concrete albeit whimsical example, here is a metaclass that rejects any class definition containing methods with mixed-case names.

In [4]:
class NoMixedCaseMeta(type):
    def __new__(cls, clsname, bases, clsdict):
        for name in clsdict:
            if name.lower() != name:
                raise TypeError('Bad attribute name: ' + name)
        return super().__new__(cls, clsname, bases, clsdict)


In [5]:
class Root(metaclass=NoMixedCaseMeta):
    pass

class A(Root):
    def foo_bar(self): # Ok
        pass


In [6]:
class B(Root):
    def fooBar(self): # TypeError
        pass


TypeError: Bad attribute name: fooBar

## Defining Classes Programmatically
You can use the function types.new_class() to instantiate new class objects.

In [8]:
def __init__(self, name, shares, price):
    self.name = name
    self.shares = shares
    self.price = price

def cost(self):
    return self.shares * self.price

cls_dict = {
    '__init__' : __init__,
    'cost' : cost,
}


In [9]:
# make a class
import types

Stock = types.new_class('Stock', (), {}, lambda ns: ns.update(cls_dict))
Stock.__module__ = __name__

In [10]:
Stock

__main__.Stock

In [12]:
Stock.__dict__

mappingproxy({'__init__': <function __main__.__init__(self, name, shares, price)>,
              'cost': <function __main__.cost(self)>,
              '__module__': '__main__',
              '__dict__': <attribute '__dict__' of 'Stock' objects>,
              '__weakref__': <attribute '__weakref__' of 'Stock' objects>,
              '__doc__': None})

In [14]:
s = Stock('ACME', 50, 91.1)
s.name, s.shares, s.price

('ACME', 50, 91.1)

## Initializing Class Members at Definition Time
Here is an example that uses this idea to create classes similar to named tuples from the collections module:

In [15]:
import operator

class StructTupleMeta(type):
    def __init__(cls, *args, **kwargs):
        super().__init__(*args, **kwargs)
        for n, name in enumerate(cls._fields):
            setattr(cls, name, property(operator.itemgetter(n)))
            

In [16]:
class StructTuple(tuple, metaclass=StructTupleMeta):
    _fields = []
    
    def __new__(cls, *args):
        if len(args) != len(cls._fields):
            raise ValueError('{} arguments required'.format(len(cls._fields)))
        return super().__new__(cls,args)


In [17]:
class Stock(StructTuple):
    _fields = ['name', 'shares', 'price']

class Point(StructTuple):
    _fields = ['x', 'y']


In [18]:
s = Stock('ACME', 50, 91.1)

In [19]:
s

('ACME', 50, 91.1)

In [20]:
s.name

'ACME'

In [21]:
s.shares, s.price

(50, 91.1)

## Implementing Multiple Dispatch with Function Annotations

In [22]:
import inspect
import types

class MultiMethod:
    '''
    Represents a single multimethod.
    '''
    def __init__(self, name):
        self._methods = {}
        self.__name__ = name
 
    def register(self, meth):
        '''
        Register a new method as a multimethod
        '''
        sig = inspect.signature(meth)
 
        # Build a type signature from the method's annotations
        types = []
        for name, parm in sig.parameters.items():
            if name == 'self':
                continue
            if parm.annotation is inspect.Parameter.empty:
                raise TypeError('Argument {} must be annotated with a type'.format(name))
            if not isinstance(parm.annotation, type):
                raise TypeError('Argument {} annotation must be a type'.format(name))
            if parm.default is not inspect.Parameter.empty:
                self._methods[tuple(types)] = meth
            types.append(parm.annotation)
            self._methods[tuple(types)] = meth
    
    def __call__(self, *args):
        '''
        Call a method based on type signature of the arguments
        '''
        types = tuple(type(arg) for arg in args[1:])
        meth = self._methods.get(types, None)
        if meth:
            return meth(*args)
        else:
            raise TypeError('No matching method for types {}'.format(types))
            
    def __get__(self, instance, cls):
        '''
        Descriptor method needed to make calls work in a class
        '''
        if instance is not None:
            return types.MethodType(self, instance)
        else:
            return self


In [23]:
class MultiDict(dict):
    '''
    Special dictionary to build multimethods in a metaclass
    '''
    def __setitem__(self, key, value):
        if key in self:
            # If key already exists, it must be a multimethod or callable
            current_value = self[key]
            if isinstance(current_value, MultiMethod):
                current_value.register(value)
            else:
                mvalue = MultiMethod(key)
                mvalue.register(current_value)
                mvalue.register(value)
                super().__setitem__(key, mvalue)
        else:
            super().__setitem__(key, value)


In [24]:
class MultipleMeta(type):
    '''
    Metaclass that allows multiple dispatch of methods
    '''
    def __new__(cls, clsname, bases, clsdict):
        return type.__new__(cls, clsname, bases, dict(clsdict))
    
    @classmethod
    def __prepare__(cls, clsname, bases):
        return MultiDict()


Which we can then use like...

In [25]:
class Spam(metaclass=MultipleMeta):
    def bar(self, x:int, y:int):
        print('Bar 1:', x, y)
 
    def bar(self, s:str, n:int = 0):
        print('Bar 2:', s, n)


In [26]:
s = Spam()

In [27]:
s.bar(1,2)

Bar 1: 1 2


In [28]:
s.bar("mystr")

Bar 2: mystr 0


There is also [functools.singledispatch](https://docs.python.org/3/library/functools.html#functools.singledispatch). Note that the dispatch happens on the type of the first argument.

In [30]:
# http://www.blog.pythonlibrary.org/2016/02/23/python-3-function-overloading-with-singledispatch/
from functools import singledispatch
 
 
@singledispatch
def add(a, b):
    raise NotImplementedError('Unsupported type')
 
 
@add.register(int)
def _(a, b):
    print("First argument is of type ", type(a))
    print(a + b)
 
 
@add.register(str)
def _(a, b):
    print("First argument is of type ", type(a))
    print(a + b)
 
 
@add.register(list)
def _(a, b):
    print("First argument is of type ", type(a))
    print(a + b)


In [31]:
add(1, 2)

First argument is of type  <class 'int'>
3


In [32]:
add("aa", "bb")

First argument is of type  <class 'str'>
aabb


In [33]:
add([1, 2], [3, 4])

First argument is of type  <class 'list'>
[1, 2, 3, 4]


## Avoiding Repetitive Property Methods

In [1]:
class Person:
    def __init__(self, name ,age):
        self.name = name
        self.age = age
 
    @property
    def name(self):
        return self._name
 
    @name.setter
    def name(self, value):
        if not isinstance(value, str):
            raise TypeError('name must be a string')
        self._name = value
 
    @property
    def age(self):
        return self._age
    
    @age.setter
    def age(self, value):
        if not isinstance(value, int):
            raise TypeError('age must be an int')
        self._age = value


Whenever you see code like this, you should explore different ways of simplifying it. One possible approach is to make a function that simply defines the property for you and returns it.

In [2]:
def typed_property(name, expected_type):
    storage_name = '_' + name
    
    @property
    def prop(self):
        return getattr(self, storage_name)
 
    @prop.setter
    def prop(self, value):
        if not isinstance(value, expected_type):
            raise TypeError('{} must be a {}'.format(name, expected_type))
        setattr(self, storage_name, value)
 
    return prop


In [3]:
class Person:
    name = typed_property('name', str)
    age = typed_property('age', int)
    def __init__(self, name, age):
        self.name = name
        self.age = age


This recipe can be tweaked in an interesting manner using the functools.partial() function.

In [4]:
from functools import partial

String = partial(typed_property, expected_type=str)
Integer = partial(typed_property, expected_type=int)

In [5]:
class Person:
    name = String('name')
    age = Integer('age')
    def __init__(self, name, age):
        self.name = name
        self.age = age


Which is looking similar to the descripter setup in 8.13.

## Defining Context Managers the Easy Way

In [6]:
import time
from contextlib import contextmanager

@contextmanager
def timethis(label):
    start = time.time()
    try:
        yield
    finally:
        end = time.time()
    print('{}: {}'.format(label, end - start))

# example use
with timethis('counting'):
    n = 1e7
    while n > 0:
        n -= 1


counting: 0.7171945571899414


In the timethis() function, all of the code prior to the yield executes as the __enter__() method of a context manager. All of the code after the yield executes as the __exit__() method. If there was an exception, it is raised at the yield statement.

In [8]:
@contextmanager
def list_transaction(orig_list):
    working = list(orig_list)
    yield working
    orig_list[:] = working  # there will only be changes to the list where there were no errors in yield


In [9]:
items = [1, 2, 3]
with list_transaction(items) as working:
    working.append(4)
    working.append(5)


In [10]:
items

[1, 2, 3, 4, 5]

In [11]:
with list_transaction(items) as working:
    working.append(10)
    raise RuntimeError

RuntimeError: 

In [12]:
items

[1, 2, 3, 4, 5]

## Executing Code with Local Side Effects

In [13]:
a = 13
exec('b = a + 1')
print(b)

14


In [1]:
def test():
    a = 13
    exec('b = a + 1')
    print(b)

test()

NameError: name 'b' is not defined

In [2]:
def test():
    a = 13
    loc = locals()
    exec('b = a + 1')
    b = loc['b']
    print(b)
    
test()

14


## Parsing and Analyzing Python Source
The ast module can be used to compile Python source code into an abstract syntax tree (AST) that can be analyzed.

In [3]:
import ast

ex = ast.parse('2 + 3*4 + x', mode='eval')
ex

In [4]:
ast.dump(ex)

"Expression(body=BinOp(left=BinOp(left=Num(n=2), op=Add(), right=BinOp(left=Num(n=3), op=Mult(), right=Num(n=4))), op=Add(), right=Name(id='x', ctx=Load())))"

## Disassembling Python Byte Code

In [5]:
def countdown(n):
    while n > 0:
        print('T-minus', n)
        n -= 1
    print('Blastoff!')


In [6]:
import dis

dis.dis(countdown)

  2           0 SETUP_LOOP              30 (to 32)
        >>    2 LOAD_FAST                0 (n)
              4 LOAD_CONST               1 (0)
              6 COMPARE_OP               4 (>)
              8 POP_JUMP_IF_FALSE       30

  3          10 LOAD_GLOBAL              0 (print)
             12 LOAD_CONST               2 ('T-minus')
             14 LOAD_FAST                0 (n)
             16 CALL_FUNCTION            2
             18 POP_TOP

  4          20 LOAD_FAST                0 (n)
             22 LOAD_CONST               3 (1)
             24 INPLACE_SUBTRACT
             26 STORE_FAST               0 (n)
             28 JUMP_ABSOLUTE            2
        >>   30 POP_BLOCK

  5     >>   32 LOAD_GLOBAL              0 (print)
             34 LOAD_CONST               4 ('Blastoff!')
             36 CALL_FUNCTION            1
             38 POP_TOP
             40 LOAD_CONST               0 (None)
             42 RETURN_VALUE


The raw byte code interpreted by the dis() function is available on functions...

In [7]:
countdown.__code__.co_code

b'x\x1e|\x00d\x01k\x04r\x1et\x00d\x02|\x00\x83\x02\x01\x00|\x00d\x038\x00}\x00q\x02W\x00t\x00d\x04\x83\x01\x01\x00d\x00S\x00'

***